In [1]:
import os, json
import pymysql
pymysql.install_as_MySQLdb()
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(os.environ['SQLALCHEMY_DATABASE_URI'])

data = {x: pd.read_sql_table(x, engine) for x in ['tool', 'parameter', 'parameter_value']}


In [2]:
data['tool'].head()

,id,tool_string,tool_name,tool_description,introduction,methods,reference,reference_link,default_selected,section_fk,input,display,result_description,video_tutorial,requires_signature
0,1,pca,PCA,Linear dimensionality reduction technique to v...,Principal Component Analysis (PCA) is a statis...,Principal Component Analysis was performed usi...,"Pearson, K. (1901). <b>LIII. On lines and plan...",https://doi.org/10.1080/14786440109462720,1,1,dataset,1,"The PCA plug-in embeds an interactive, three-d...",https://www.youtube.com/watch?v=r7WYtxeAaDw,0
1,2,tsne,t-SNE,Nonlinear dimensionality reduction technique t...,t-distributed Stochastic Neighbor Embedding (t...,t-SNE analysis was performed using the `TSNE` ...,L.J.P. van der Maaten and G.E. Hinton. <b>Visu...,https://lvdmaaten.github.io/publications/paper...,0,1,dataset,0,,None,0
2,3,clustergrammer,Clustergrammer,Interactive hierarchical clustering heatmap vi...,Clustergrammer is a web-based tool for visuali...,The interactive heatmap was generated using Cl...,"Fernandez, N.F., Gundersen, G.W., Rahman, A., ...",http://dx.doi.org/10.1038/sdata.2017.151,1,1,dataset,1,The Clustergrammer plug-in embeds an interacti...,https://youtu.be/8MgvyeroW98?t=59s,0
3,4,library_size_analysis,Library Size Analysis,Analysis of readcount distribution for the sam...,In order to quantify gene expression in an RNA...,Read counts were calculated by performing the ...,,,1,1,dataset,1,The Library Size Analysis plug-in embeds an in...,None,0
4,5,signature_table,Differential Expression Table,Differential expression analysis between two g...,Gene expression signatures are alterations in ...,The gene expression signature was generated by...,,,0,2,signature,1,The Signature Table plug-in embeds an interact...,None,1


In [3]:
data['parameter'].head()

,id,parameter_name,parameter_description,parameter_string,tool_fk
0,2,Top Genes,Number of most variably expressed genes to use...,nr_genes,1
1,3,Normalization,Normalization method for the dataset,normalization,1
2,5,Top Genes,Number of most variably expressed genes to use...,nr_genes,2
3,6,Normalization,Normalization method for the dataset,normalization,2
4,7,Top Genes,Number of most variably expressed genes to use...,nr_genes,3


In [4]:
data['parameter_value'].head()

,id,parameter_fk,value,type,default
0,3,2,500,int,0
1,4,2,2500,int,1
2,5,2,5000,int,0
3,6,3,logCPM,str,1
4,7,3,quantile,str,0


In [18]:
pv = {x: data['parameter_value'][data['parameter_value']['parameter_fk'] == x].drop(['parameter_fk', 'type', 'id'], axis=1).to_dict(orient='records') for x in data['parameter_value']['parameter_fk']}
pv

{2: [{'default': 0, 'value': '500'},
  {'default': 1, 'value': '2500'},
  {'default': 0, 'value': '5000'}],
 3: [{'default': 1, 'value': 'logCPM'},
  {'default': 0, 'value': 'quantile'},
  {'default': 0, 'value': 'VST'}],
 5: [{'default': 0, 'value': '500'},
  {'default': 1, 'value': '2500'},
  {'default': 0, 'value': '5000'}],
 6: [{'default': 1, 'value': 'logCPM'},
  {'default': 0, 'value': 'quantile'},
  {'default': 0, 'value': 'VST'}],
 7: [{'default': 0, 'value': '500'},
  {'default': 1, 'value': '2500'},
  {'default': 0, 'value': '5000'}],
 8: [{'default': 1, 'value': 'logCPM'},
  {'default': 0, 'value': 'quantile'},
  {'default': 0, 'value': 'VST'}],
 9: [{'default': 0, 'value': '100'},
  {'default': 0, 'value': '250'},
  {'default': 1, 'value': '500'}],
 10: [{'default': 1, 'value': 'True'}, {'default': 0, 'value': 'False'}],
 11: [{'default': 1, 'value': 'True'}, {'default': 0, 'value': 'False'}],
 12: [{'default': 1, 'value': 'True'}, {'default': 0, 'value': 'False'}],
 13: [

In [20]:
data['parameter']['values'] = [pv.get(x, '') for x in data['parameter']['id']]

In [22]:
p = {x: data['parameter'][data['parameter']['tool_fk'] == x].drop(['tool_fk', 'id'], axis=1).to_dict(orient='records') for x in data['parameter']['tool_fk']}
p

{1: [{'parameter_description': 'Number of most variably expressed genes to use for the analysis',
   'parameter_name': 'Top Genes',
   'parameter_string': 'nr_genes',
   'values': [{'default': 0, 'value': '500'},
    {'default': 1, 'value': '2500'},
    {'default': 0, 'value': '5000'}]},
  {'parameter_description': 'Normalization method for the dataset',
   'parameter_name': 'Normalization',
   'parameter_string': 'normalization',
   'values': [{'default': 1, 'value': 'logCPM'},
    {'default': 0, 'value': 'quantile'},
    {'default': 0, 'value': 'VST'}]},
  {'parameter_description': 'Whether to perform Z-score on the rows of the normalized dataset',
   'parameter_name': 'Z-score',
   'parameter_string': 'z_score',
   'values': [{'default': 1, 'value': 'True'},
    {'default': 0, 'value': 'False'}]}],
 2: [{'parameter_description': 'Number of most variably expressed genes to use for the analysis',
   'parameter_name': 'Top Genes',
   'parameter_string': 'nr_genes',
   'values': [{'defa

In [23]:
data['tool']['parameters'] = [p.get(x, []) for x in data['tool']['id']]
data['tool'].head()

,id,tool_string,tool_name,tool_description,introduction,methods,reference,reference_link,default_selected,section_fk,input,display,result_description,video_tutorial,requires_signature,parameters
0,1,pca,PCA,Linear dimensionality reduction technique to v...,Principal Component Analysis (PCA) is a statis...,Principal Component Analysis was performed usi...,"Pearson, K. (1901). <b>LIII. On lines and plan...",https://doi.org/10.1080/14786440109462720,1,1,dataset,1,"The PCA plug-in embeds an interactive, three-d...",https://www.youtube.com/watch?v=r7WYtxeAaDw,0,"[{'parameter_name': 'Top Genes', 'parameter_de..."
1,2,tsne,t-SNE,Nonlinear dimensionality reduction technique t...,t-distributed Stochastic Neighbor Embedding (t...,t-SNE analysis was performed using the `TSNE` ...,L.J.P. van der Maaten and G.E. Hinton. <b>Visu...,https://lvdmaaten.github.io/publications/paper...,0,1,dataset,0,,None,0,"[{'parameter_name': 'Top Genes', 'parameter_de..."
2,3,clustergrammer,Clustergrammer,Interactive hierarchical clustering heatmap vi...,Clustergrammer is a web-based tool for visuali...,The interactive heatmap was generated using Cl...,"Fernandez, N.F., Gundersen, G.W., Rahman, A., ...",http://dx.doi.org/10.1038/sdata.2017.151,1,1,dataset,1,The Clustergrammer plug-in embeds an interacti...,https://youtu.be/8MgvyeroW98?t=59s,0,"[{'parameter_name': 'Top Genes', 'parameter_de..."
3,4,library_size_analysis,Library Size Analysis,Analysis of readcount distribution for the sam...,In order to quantify gene expression in an RNA...,Read counts were calculated by performing the ...,,,1,1,dataset,1,The Library Size Analysis plug-in embeds an in...,None,0,[]
4,5,signature_table,Differential Expression Table,Differential expression analysis between two g...,Gene expression signatures are alterations in ...,The gene expression signature was generated by...,,,0,2,signature,1,The Signature Table plug-in embeds an interact...,None,1,[]


In [25]:
d = data['tool'].drop(['id'], axis=1).to_dict(orient='records')
d

[{'default_selected': 1,
  'display': 1,
  'input': 'dataset',
  'introduction': 'Principal Component Analysis (PCA) is a statistical technique used to identify global patterns in high-dimensional datasets. It is commonly used to explore the similarity of biological samples in RNA-seq datasets. To achieve this, gene expression values are transformed into Principal Components (PCs), a set of linearly uncorrelated features which represent the most relevant sources of variance in the data, and subsequently visualized using a scatter plot.',
  'methods': 'Principal Component Analysis was performed using the PCA function from in the sklearn Python module. Prior to performing PCA, the raw gene counts were normalized using the {normalization} method, filtered by selecting the {nr_genes} genes with most variable expression, and finally transformed using the Z-score method.',
  'parameters': [{'parameter_description': 'Number of most variably expressed genes to use for the analysis',
    'param

In [26]:
with open('tools.json', 'w') as openfile:
    openfile.write(json.dumps(d, indent=4))

In [32]:
import os

In [46]:
os.system('rm -r img')
for tool in data['tool']['tool_string'].tolist():
    os.system('mkdir -p img/'+tool)
    os.system('touch img/'+tool+'/'+tool+'-icon.png')
    os.system('touch img/'+tool+'/'+tool+'-example.png')

In [40]:
tool

'epigenomics_roadmap_enrichment'

In [41]:
%ls

Untitled.ipynb  tools.json      tools.py
